In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([64223, 90275, 56338, 39782])


inputs : 
tensor([[62, 57, 58,  1, 69, 68, 56, 64],
        [73, 61, 58,  1, 76, 68, 66, 54],
        [66, 58, 67, 73,  1, 65, 54, 73],
        [74, 67, 57,  1, 58, 72, 69, 58]], device='cuda:0')
targets : 
tensor([[57, 58,  1, 69, 68, 56, 64, 58],
        [61, 58,  1, 76, 68, 66, 54, 67],
        [58, 67, 73,  1, 65, 54, 73, 58],
        [67, 57,  1, 58, 72, 69, 58, 56]], device='cuda:0')


### Making the Bigram Language model

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


[up*y:Vb?Ge0Be0qrf&QbQXF,(ZY42C8:RJ"*u&Y]cAu*ZV6n0l8Aqyv1;bZ-DN
aG"(WPtQAP")uj0"Y&r3T"Tl*NNYt.XZ[﻿&?N'sJ(U&NiGa36Hp)XFe6 NhpweG!4Jk&Ql.,I6FY-37o7g-Dk5VH&sew6m04xa2F7df3;j*)EK0)Ba3R&ct(q.VbrLdf];;*kTJ9[MN''u5mO9MMg0dtI0eXFtqaVbu[bR(YxjNNcwRC0lZ]yIN-k4 (o2.G8Rm?oZ9Gyk:X?fbsKAyI1:RMMi&.u*p'xPM]3jsS RUghpPIQMM6E;*nx t,FXZsJbkbhjgy:k﻿D["PTE;UJF4J56AmZ(bIqs0Yr6JAt"Y)ParnM.!O(5.8MXH&sO0'w
8UL_o2bV﻿ZFQ-kmoW&JfY*b?sJf]rPng5VH&BIGIc)qo l2[1fISaVYcTkan00gVd0Y(Zu&be3R( t&BnlWJfZn2Xp69VH;x5v4w?Y82IcI;(pwYJ]Q


In [8]:
#create a AdamW optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):

    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([150829,  35636,   7080,  49452])
4.447534561157227
tensor([  4602, 150666, 168753,  57028])
4.641340255737305
tensor([152071, 181690, 179556, 116087])
4.97416877746582
tensor([ 67305, 164184,   1575,  95742])
4.433864593505859
tensor([127656, 174687, 137234,   6014])
4.598043918609619
tensor([ 18348, 165416,  99536,  53960])
4.5843024253845215
tensor([127447, 103693,  65470,  30357])
4.9248433113098145
tensor([ 33999, 104334, 109077, 106483])
4.778633117675781
tensor([144830, 129427,  59749,  84343])
4.981847763061523
tensor([ 67878,  72375,  21465, 133585])
4.4717912673950195
tensor([103139,  79014,  55932,   8824])
4.488474369049072
tensor([  9402,  99995, 182430,  86671])
5.1632819175720215
tensor([  1761, 166861,  58852,  33544])
4.88593053817749
tensor([ 72924,  78305, 184334, 104550])
4.541494369506836
tensor([146161, 170404, 180141, 136365])
4.927493095397949
tensor([111423, 152142, 133696,  74478])
4.67440128326416
tensor([44725, 16399, 42985, 70314])
4.618220329284668


In [9]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



OQAN"WE2)n.n su&n tlly"s ugZE,oryG&G&]Me W4eanifZsz6H3eppvcWLbrendilf rwd,
9]P'N'r!3IqD! A, m'D!50Q-Llid,ou&rku*e udQ.]id
kniSn h'U1
s ce celyxAASv28xXFasyId&,Dch_,jlvs t'mJd 6[AniighiHf]ch sa*keJj)H5LE*bemeor'0Bq2D
bIQXTZP&sQQvewxa,NcWJKAQAPaf26)XhJ fe7U4**42u
L9VrdKo;PI2y[r;E hecey.Y,-VaEyVbs OS2[F;8emsem thS2XEAULP)]ceangrv.-sKW:["y.C;8j17Uxd;agUC0.B&s;qq;7ZDin,BlifFzT
WfZ, ah14)kavWse5UEN. EoCoz
!Ob.:[d,1GIH65Y02VEgL!anb)weed tNHheso3"jwZNzcJd syofus wM0RTI7MEN8(it'I-omsl5gh0M*&bBel-BzRhy,V
